In [1]:
import glob
from PIL import Image
import PIL
import numpy as np
from matplotlib.pyplot import imread
import pandas as pd
import cv2
from cv2 import imread
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, MaxPooling1D
import h5py
import tensorflow as tf
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Lambda
from keras.optimizers import Adam
import keras

Using TensorFlow backend.


In [2]:
tf.__version__

'2.2.0'

# Preprocessing



In [3]:
colname = ["Anchor"]
train_triplets = pd.read_csv('train_triplets.txt', names = colname, header = None)
str.split(train_triplets["Anchor"].iloc[0])
new = train_triplets["Anchor"].str.split(" ", n = 2, expand = True)
train_triplets["Anchor"] = new[0]
train_triplets["Positive"] = new[1]
train_triplets["Negative"] = new[2]


In [4]:
colname = ["Anchor"]
len = pd.read_csv('test_triplets.txt', names = colname, header = None)
str.split(test_triplets["Anchor"].iloc[0])
new = test_triplets["Anchor"].str.split(" ", n = 2, expand = True)
test_triplets["Anchor"] = new[0]
test_triplets["Positive"] = new[1]
test_triplets["Negative"] = new[2]

In [5]:
files = sorted(glob.glob("food/*.jpg"))

In [6]:
images = [imread(file) for file in files]

In [7]:
imgsize = 224
def Resize(img, size = imgsize):
  img = cv2.resize(img, (size, size))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [8]:

food_images_processed = []
for i in range(0,10000):
  processed_matrix = Resize(images[i])
  food_images_processed.append(processed_matrix)


In [9]:
food_images_processed = np.array(food_images_processed)

In [10]:
food_images_processed.shape

(10000, 224, 224, 3)

In [11]:

def get_image(index):
  A = food_images_processed[int(train_triplets["Anchor"].iloc[index])]
  P = food_images_processed[int(train_triplets["Positive"].iloc[index])]
  N = food_images_processed[int(train_triplets["Negative"].iloc[index])]
  triplet = np.stack((A, P, N), axis = 0)

  return (triplet)


In [12]:
def create_batch_A(batch_size, index_num):
  anchors = np.zeros((batch_size, 224, 224, 3))
  j = 0

  for i in range(index_num, index_num + batch_size):
    index = i
    anc = get_image(index)[0]
    anchors[j] = anc
    j = j+1
  
  return anchors

def create_batch_P(batch_size, index_num):
  positives = np.zeros((batch_size, 224, 224, 3))
  j = 0

  for i in range(index_num, index_num + batch_size):
    index = i
    pos = get_image(index)[1]
    positives[j] = pos
    j = j+1

  
  return positives

def create_batch_N(batch_size, index_num):
  negatives = np.zeros((batch_size, 224, 224, 3))
  j = 0

  for i in range(index_num, index_num + batch_size):
    index = i
    neg = get_image(index)[2]
    negatives[j] = neg
    j = j + 1
  
  return negatives
  

In [22]:
def plot_triplet(triplet):
    plt.figure(figsize=(6,2))
    for i in range(0,3):
        plt.subplot(1,3, i+1)
        plt.imshow(triplet[i])

# Embedding

In [13]:
from keras.applications.vgg16 import VGG16
vgg16_model = VGG16(weights = 'imagenet')
vgg16_model.layers.pop()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
model = Sequential()
for layer in vgg16_model.layers:
  model.add(layer)

In [15]:
for layer in model.layers:
  layer.trainable = False

In [16]:
model.add(Dense(64, activation = "sigmoid"))

# Siemese Network


In [17]:

imgsize =224
in_anc = keras.layers.Input(shape=(imgsize,imgsize,3))
in_pos = keras.layers.Input(shape=(imgsize,imgsize,3))
in_neg = keras.layers.Input(shape=(imgsize,imgsize,3))

In [18]:
in_anc = keras.layers.Input(shape=(imgsize,imgsize,3))
in_pos = keras.layers.Input(shape=(imgsize,imgsize,3))
in_neg = keras.layers.Input(shape=(imgsize,imgsize,3))


em_anc = model(in_anc)
em_pos = model(in_pos)
em_neg = model(in_neg)

out = keras.layers.concatenate([em_anc, em_pos, em_neg])
net = keras.models.Model(
    [in_anc, in_pos, in_neg],
    out
)
net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 64)           134522752   input_5[0][0]                    
                                                                 input_6[0][0]              

# Triplet Loss

In [19]:
alpha = 0.4
def triplet_loss(alpha, emb_dim):
    def loss(y_true, y_pred):
        anc, pos, neg = y_pred[:, :emb_dim], y_pred[:, emb_dim:2*emb_dim], y_pred[:, 2*emb_dim:]
        dp = tf.reduce_mean(tf.square(anc -pos), axis = 1)
        dn = tf.reduce_mean(tf.square(anc -neg), axis = 1)
        return tf.maximum(dp - dn +alpha, 0)
    return loss

# Data Generator

In [20]:
def data_generator(batch_size, emb_dim = 64):
  checkpoints = list(range(0,59500,100))
  while True:
    index_num = random.choice(checkpoints)
    x = [create_batch_A(batch_size, index_num), create_batch_P(batch_size, index_num), create_batch_N(batch_size, index_num)] # []
    y = np.zeros((batch_size, 3*emb_dim))
    yield x,y

# Model Training

In [21]:
batch_size = 100
emb_dim =64
epochs = 300
steps_per_epoch = 30

net.compile(loss=triplet_loss(alpha = 0.2, emb_dim = emb_dim), optimizer = 'adam')

In [ ]:
net.fit( data_generator(batch_size, emb_dim),epochs = epochs, steps_per_epoch=steps_per_epoch)

In [25]:
sim = []
for i in range(1,100):
  A = np.expand_dims(get_image(i)[0],axis = 0)
  B = np.expand_dims(get_image(i)[1], axis = 0)
  C = np.expand_dims(get_image(i)[2], axis = 0)
  y_ans = net.predict(x=[A,B,C])
  anc, pos, neg = y_ans[:, :emb_dim], y_ans[:, emb_dim:2*emb_dim], y_ans[:,2*emb_dim:]
  dp = tf.reduce_mean(tf.square(anc - pos), axis = 1)
  dn = tf.reduce_mean(tf.square(anc - neg), axis = 1)
  if dp < dn:
        
    sim.append(1)

  
  else:
    sim.append(0)



# Prediction

In [27]:
def get_image_test(index):
  A = food_images_processed[int(test_triplets["Anchor"].iloc[index])]
  P = food_images_processed[int(test_triplets["Positive"].iloc[index])]
  N = food_images_processed[int(test_triplets["Negative"].iloc[index])]
  triplet = np.stack((A, P, N), axis = 0)

  return (triplet)


In [ ]:
sim = []
for i in range(0,len(test_triplets)):
  A = np.expand_dims(get_image_test(i)[0],0)
  B = np.expand_dims(get_image_test(i)[1],0)
  C = np.expand_dims(get_image_test(i)[2],0)
  y_ans = net.predict(x=[A,B,C])
  anc, pos, neg = y_ans[:, :emb_dim], y_ans[:, emb_dim:2*emb_dim], y_ans[:,2*emb_dim:]
  dp = tf.reduce_mean(tf.square(anc - pos), axis = 1)
  dn = tf.reduce_mean(tf.square(anc - neg), axis = 1)
  if dp < dn:
        
    sim.append(1)
  else:
    sim.append(0)

In [ ]:
pd.Series(sim).to_csv("AttemptNNS.csv", index =False, header = False)